## Get hausdorff distance for models1, models2

In [72]:
import os
import colorcet as cc
from colour import Color

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np

import pymeshlab as ml

import pyvista as pv
from pyvista import examples


from collections import defaultdict

from colorspacious import cspace_converter

In [74]:
pv.global_theme.transparent_background = True

pre = './201818/'

#---------convert files to .stl-----------------
# tmp = os.listdir()

# import meshio

# fnames = ['pialnn/pp.obj','pialnn/tp.obj','vox2cortex/pp.ply','vox2cortex/pw.ply','vox2cortex/tp.ply','vox2cortex/tw.ply']
# for fname in fnames:
#     mesh = meshio.read(pre+fname)
#     meshio.write(pre+fname[:-4]+'.stl', mesh, file_format="stl")

models1 = ['cortexode','corticalflow','deepcsr','pialnn','vox2cortex']
models2 = ['cortexode','corticalflow','deepcsr','topofit','vox2cortex']
fs = ['tp.stl', 'pp.stl', 'tw.stl', 'pw.stl']
file_truths1 = [pre+'./'+m+'/'+fs[0] for m in models1]
files1 = [pre+'./'+m+'/'+fs[1] for m in models1]

file_truths2 = [pre+'./'+m+'/'+fs[2] for m in models2]
files2 = [pre+'./'+m+'/'+fs[3] for m in models2]

def calc_distance():
    ms = ml.MeshSet()
    ms.load_new_mesh(file_truth)
    ms.load_new_mesh(file)
    ms.compute_scalar_by_distance_from_another_mesh_per_vertex()
    ms.compute_color_from_scalar_per_vertex()
    return ms

def calc_hausdorff_distance(file_truth, file):
    ms = ml.MeshSet()
    ms.load_new_mesh(file_truth)
    ms.load_new_mesh(file)
    d = ms.get_hausdorff_distance()
    return d 

def get_hdd(files,file_truths):
    tmps = []
    for i in range(len(files)):
        file_truth = file_truths[i]
        file = files[i]
        d = calc_hausdorff_distance(file_truth, file)
        tmps.append(d)
    return tmps

## Get the Coloring Values: tmp1, tmp2

In [75]:
def calculate_tmp(tmp, d, attr,way='william'):
    if way=='william':
        return [abs(j)/d[attr]*100 for j in tmp] # normalize with diag_mesh distance
    return [abs(j) for j in tmp]


def truncate_tmp(tmp,lower_percentile=5, higher_percentile = 95):   
    print('real max of tmp: ',max(tmp))
    minz = np.percentile(tmp, lower_percentile)
    maxz = np.percentile(tmp, higher_percentile)
    # truncate it between minz and maxz
    tmp = [max(max(i, minz),min(i,maxz)) for i in tmp]
    return tmp

def maximize_tmp(mses, tmps, maximize=True, MAX_VALUE = 1.8):
    if maximize:
        for i in range(len(tmps)):
            # modify tmp value to include the same MAX_VALUE as its max_value
            tmps[i] = [min(j, MAX_VALUE) for j in tmps[i]]
            idx = tmps[i].index(max(tmps[i]))
            tmps[i][idx] = MAX_VALUE
            # assign tmp values to mesh
            mses[i]['values'] = tmps[i]
    return mses, tmps
    

def getit(files,file_truths,d,attr,MAX_VALUE=4.5):
    tmps = []
    mses = []
    for i in range(len(files)):
#         plt.subplot(5,1,i+1)
        file_truth = file_truths[i]
        file = files[i]
        ms2 = ml.MeshSet()
        ms2.load_new_mesh(file_truth)
        ms2.load_new_mesh(file)
        ms2.compute_scalar_by_distance_from_another_mesh_per_vertex()
        ms2.compute_color_from_scalar_per_vertex()
        ms2.apply_color_inverse_per_vertex()

        tmp = list(ms2[1].vertex_scalar_array())
        tmp = calculate_tmp(tmp,d[i], attr)
        tmp = truncate_tmp(tmp)
        tmps.append(tmp)
        
        mesh = pv.read(file)
        mses.append(mesh)
        
    mses,tmps = maximize_tmp(mses,tmps)
    return mses, tmps

In [76]:
d1 = get_hdd(files1, file_truths1)
d2 = get_hdd(files2, file_truths2)

In [77]:
attr = 'diag_mesh_0'
mses1, tmps1 = getit(files1,file_truths1,d1,attr)
mses2, tmps2 = getit(files2,file_truths2,d2,attr)

real max of tmp:  6.04252876311079
real max of tmp:  2.099364022693511
real max of tmp:  2.169705077863622
real max of tmp:  1.7714791422515495
real max of tmp:  1.2495406642202718
real max of tmp:  6.180894003732273
real max of tmp:  3.4294845901979683
real max of tmp:  4.128479694645356
real max of tmp:  1.3262658326035814
real max of tmp:  1.9227369635716376


## Visualize the distance/diag_mesh_0

In [ ]:
def plt_tmp(tmps,suptitle,models):
    plt.figure(figsize=(15,4))
    plt.suptitle(suptitle)
    for i in range(1,6):
        plt.subplot(1,5,i)
        data = tmps[i-1]
        plt.hist(data,bins = 30, weights=np.ones_like(data) / len(data))
        plt.xlim(-0.5,3)
        plt.ylim(0,1)
        plt.title(models[i-1])
        plt.xlabel('distance/diag (%)')
        plt.ylabel('vertex counts Frequency')
        
# plt_tmp(tmps1,'pial',models1)
# plt_tmp(tmps2,'white',models2)

In [ ]:
plt_tmp(tmps1,'pial',models1)

In [ ]:
plt_tmp(tmps2,'white',models2)

In [ ]:
for i in tmps1:
    print(np.mean(i),min(i),max(i))

## Visualize brain and save as PNG

### Play with ColorMap

In [ ]:
cmap=mpl.colormaps['inferno']

In [ ]:
cmap

#### Linear Colors 

In [ ]:
cmap = mpl.colormaps['inferno']

In [ ]:
n = len(cmap.colors)
cs = [36, 72, 109, 145, 182, 218, 255]
colors = [cmap.colors[i] for i in cs]
colors = [Color(rgb=i) for i in colors]

In [ ]:
def getcolormap(colors = colors):
    c = colors
    ccc = defaultdict(list)
    ccc[1] = list(c[0].range_to(c[4],200))
    ccc[7] = list(c[4].range_to(c[-1],56))

    cccs = []
    for k in range(1,8):
        ccc[k] = [list(i.get_rgb())+[1] for i in ccc[k]]
        cccs += ccc[k]

    # Define the colors we want to use
    colors = cccs
    my_colormap = ListedColormap(np.array(colors))
    return my_colormap

def seeit(mesh,):
    my_colormap = getcolormap()
    # Make the colormap from the listed colors
    
    pl = pv.Plotter()
    _ = pl.add_mesh(mesh,cmap=my_colormap)
    pl.set_viewup([0,1,0])
    pl.show()

In [ ]:
seeit(mses1[0])

#### Other ways

In [ ]:
cmap = mpl.colormaps['inferno']

In [ ]:
cmap

In [ ]:
newcolors = []

for i in range(66):
    newcolors.append(cmap.colors[i])
    newcolors.append(cmap.colors[i])
    newcolors.append(cmap.colors[i])
        
for i in range(80,253):
    if i % 3 == 0:
        newcolors.append(cmap.colors[i])
        


In [ ]:
len(newcolors)

In [ ]:
my_colormap = ListedColormap(np.array(newcolors[::-1]))

### Visualize it

In [ ]:
cmap_candidates = [mpl.colormaps['inferno'].reversed(), my_colormap,mpl.colormaps['inferno'] ]
def visualize_brain(m = 'pial', models = models1, mses = mses1, scale = 6, cmap_idx = 0):
    cmap = cmap_candidates[i]
    for i in range(len(models)):
        viewup1 = [1,0,0]
        viewup2 = [0,0,1]
        mesh = mses[i]
        pl = pv.Plotter()
        actor = pl.add_mesh(mesh,cmap=cmap)

        pl.set_background('white', all_renderers=False)
        pl.remove_scalar_bar()

        pl.view_yz()
        pl.set_viewup(viewup1)
        if i==3 and m == 'pial':
            pl.set_viewup(viewup2)

        pl.remove_legend()
        pl.remove_floors()
        pl.remove_bounding_box()
        pl.remove_bounds_axes()
        pl.update_bounds_axes()
        pl.show()
        pl.screenshot(m + models[i] + '.png', transparent_background=True, return_img=True, scale=scale)

In [ ]:
visualize_brain()

In [ ]:
visualize_brain(m = 'white', models = models2, mses = mses2, scale = 6)

### Get Legend Plot

In [92]:
cmap = mpl.colormaps['inferno'].reversed()

viewup1 = [1,0,0]
pv.global_theme.font.color = 'black'
mesh = mses1[0]
pl = pv.Plotter()
actor = pl.add_mesh(mesh,cmap=cmap)

pl.set_background('white', all_renderers=False)
# pl.remove_scalar_bar()

pl.view_yz()
pl.set_viewup(viewup1)
# pl.remove_actor(actor)
# pl.remove_legend()
pl.remove_floors()
pl.remove_bounding_box()
# pl.remove_bounds_axes()
# pl.update_bounds_axes()
pl.show()

2023-04-13 00:41:10.120 ( 711.531s) [          E992E1]        vtkRenderer.cxx:1172  WARN| vtkOpenGLRenderer (0x7fae4144ea00): Resetting view-up since view plane normal is parallel


Widget(value="<iframe src='http://localhost:60323/index.html?ui=P_0x7fae1fac0fa0_7&reconnect=auto' style='widt…

In [94]:
pl.screenshot('legend.png', transparent_background=True, return_img=True, scale=6)

pyvista_ndarray([[[255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  ...,
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0]],

                 [[255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  ...,
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0]],

                 [[255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  ...,
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0]],

                 ...,

                 [[255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  ...,
                  [255, 255, 255,   0],

In [89]:
dir(pl)

['_BasePlotter__before_close_callback',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_legend_label',
 '_before_close_callback',
 '_check_has_ren_win',
 '_check_rendered',
 '_clear_ren_win',
 '_closed',
 '_datasets',
 '_first_time',
 '_has_background_layer',
 '_id_name',
 '_image_depth_null',
 '_image_scale',
 '_initialized',
 '_on_first_render_request',
 '_on_render_callbacks',
 '_picked_block_index',
 '_picked_mesh',
 '_picked_point',
 '_picker',
 '_picking_left_clicking_observer',
 '_picking_text',
 '_prep_for_close',
 '_rendered',
 '_save_image',
 '_save_panel',
 '_scalar_bar_slot_lookup',
 '_scalar_bar_slots',
 '_scalar_bars',
 '_suppress_render

## Put Multiple Figures in One Using Matplotlib

### Get all the PNG files

In [ ]:


pialfiles = []
whitefiles = []

all_files = os.listdir('./whitepng')
for f in all_files:
    if f[:4] == 'pial' and f[-4:] == '.png' and '_' not in f:
        pialfiles.append(f)
    if f[:5] == 'white' and f[-4:] == '.png' and '_' not in f:
        whitefiles.append(f)  
        
all_files = os.listdir('./pialpng')
for f in all_files:
    if f[:4] == 'pial' and f[-4:] == '.png' and '_' not in f:
        pialfiles.append(f)
    if f[:5] == 'white' and f[-4:] == '.png' and '_' not in f:
        whitefiles.append(f) 

In [ ]:
pialfiles

### Convert PNG to SVG

In [ ]:
from PIL import Image
import os

# Set the file paths

png_file_path = './whitepng/whitevox2cortex.png'
svg_file_path = ''

# Open the PNG image
img = Image.open(png_file_path)

# Convert the image to SVG format
img.save(svg_file_path, 'svg')

In [ ]:
from PIL import Image
import svgwrite

# Load the PNG image
im = Image.open("./whitepng/whitevox2cortex.png")

# Create a new SVG drawing
svg = svgwrite.Drawing("image.svg")

# Create an SVG image object
svg_image = svgwrite.image.Image("data:image/png;base64," + im.tobytes().decode("base64").encode("base64").replace("\n", ""))

# Add the image object to the SVG drawing
svg.add(svg_image)

# Save the SVG drawing
svg.save()

### modify SVG file to remove reduplicated ids

In [35]:
import random

#read input file
def modify_svg(svgfile, savetofile):
    fin = open(svgfile, "rt")

    #read file contents to string
    data = fin.read()
    
    #replace all occurrences of the required string
    
    # generate random str
    x = random.random()
    x = int(x*100000)
    data = data.replace('id="img', 'id="img'+str(x))
    data = data.replace('#img', '#img'+str(x))
    data = data.replace('id="clip', 'id="clip'+str(x))
    data = data.replace('#clip', '#clip'+str(x))    
    
    #close the input file
    fin.close()

    #open the input file in write mode
    fin = open('modified_' + svgfile, "wt")

    #overrite the input file with the resulting data
    fin.write(data)

    #close the file
    fin.close()

### Put the files together in one plot

In [ ]:
whitefiles

In [ ]:
#import libraries
import cv2
  
# create figure
fig = plt.figure(figsize=(10, 7))
  
# setting values to rows and column variables
rows = 2
columns = 2
  
# reading images
Image1 = cv2.imread(whitefiles[0])
Image2 = cv2.imread(whitefiles[1])
Image3 = cv2.imread(whitefiles[2])
Image4 = cv2.imread(whitefiles[3])
  
# Adds a subplot at the 1st position
fig.add_subplot(rows, columns, 1)
  
# showing image
plt.imshow(Image1)
plt.axis('off')
plt.title("First")
  
# Adds a subplot at the 2nd position
fig.add_subplot(rows, columns, 2)
  
# showing image
plt.imshow(Image2)
plt.axis('off')
plt.title("Second")
  
# Adds a subplot at the 3rd position
fig.add_subplot(rows, columns, 3)
  
# showing image
plt.imshow(Image3)
plt.axis('off')
plt.title("Third")
  
# Adds a subplot at the 4th position
fig.add_subplot(rows, columns, 4)
  
# showing image
plt.imshow(Image4)
plt.axis('off')
plt.title("Fourth")

In [ ]:



def getcolormap(c1,c2):
    green = Color(c1)
    red = Color(c2)
    c = list(green.range_to(red,8))

    ccc = defaultdict(list)
    ccc[1] = list(green.range_to(c[1],3))
    ccc[2] = list(c[1].range_to(c[2],6))
    ccc[3] = list(c[2].range_to(c[3],10))
    ccc[4] = list(c[3].range_to(c[4],20))
    ccc[5] = list(c[4].range_to(c[5],40))
    ccc[6] = list(c[5].range_to(c[6],80))
    ccc[7] = list(c[6].range_to(c[7],96))

    cccs = []
    for k in range(1,8):
        ccc[k] = [list(i.get_rgb())+[1] for i in ccc[k]]
        cccs += ccc[k]

    # Define the colors we want to use
    colors = cccs
    my_colormap = ListedColormap(np.array(colors))
    return my_colormap

def seeit(mesh,c1='lightgreen',c2='lightpink'):
    my_colormap = getcolormap(c1,c2)
    # Make the colormap from the listed colors
    
    pl = pv.Plotter(shape=(1,5))
    pl.subplot(0,1)
    _ = pl.add_mesh(mesh,cmap=my_colormap)
    pl.set_viewup([0,1,0])
    pl.show()

## Exploring cmap

In [ ]:


cmaps = {}

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))


def plot_color_gradients(category, cmap_list):
    # Create figure and adjust figure height to number of colormaps
    nrows = len(cmap_list)
    figh = 0.35 + 0.15 + (nrows + (nrows - 1) * 0.1) * 0.22
    fig, axs = plt.subplots(nrows=nrows + 1, figsize=(6.4, figh))
    fig.subplots_adjust(top=1 - 0.35 / figh, bottom=0.15 / figh,
                        left=0.2, right=0.99)
    axs[0].set_title(f'{category} colormaps', fontsize=14)

    for ax, name in zip(axs, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=mpl.colormaps[name])
        ax.text(-0.01, 0.5, name, va='center', ha='right', fontsize=10,
                transform=ax.transAxes)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axs:
        ax.set_axis_off()

    # Save colormap list for later.
    cmaps[category] = cmap_list
    return cmaps
    
cmaps = plot_color_gradients('Perceptually Uniform Sequential',
                     ['viridis', 'plasma', 'inferno', 'magma', 'cividis'])

In [ ]:
mpl.colormaps['viridis']

In [ ]:
dir(pl)

In [ ]:
help(pl.add_mesh)

In [ ]:
help(pv.Plotter())

## plot brain and Save it as PNG

### Clip the data "outliers"

In [ ]:
i = 0
mesh = mses1[0]
pl = pv.Plotter()
actor = pl.add_mesh(mesh,cmap=mpl.colormaps['inferno'])

pl.set_background('white', all_renderers=False)
pl.remove_scalar_bar()
pl.set_viewup([0,1,0])
pl._image_scale = 2
if i == 3:
    pl.view_xz()
pl.remove_legend()
pl.remove_floors()
pl.remove_bounding_box()
pl.remove_bounds_axes()
pl.update_bounds_axes()
pl.show()
pl.close()

## Pl save graphic

In [ ]:
pl.save_graphic(m + '_' + models[i] + '_' + '.svg')

## PL save as HTML

In [ ]:
from potrace import Bitmap

# Initialize data, for example convert a PIL image to a numpy array
# [...]

bitmap = Bitmap(data)
path = bitmap.trace()

In [ ]:
pl.save_graphic('test.svg', "", raster=False)

In [ ]:
pl.export_obj('try.obj')

In [ ]:
# Test to get model 4 position.


m = ''

for i in range(5):
    pl = pv.Plotter()
    mesh = mses1[i]
    actor = pl.add_mesh(mesh,cmap=mpl.colormaps['inferno'])
    pl.set_background('white', all_renderers=False)
    pl.remove_scalar_bar()
    pl.set_viewup([0,1,0])
    pl._image_scale = 2
    if i == 3:
        pl.view_xz()
    pl.remove_legend()
    pl.remove_floors()
    pl.remove_bounding_box()
    pl.remove_bounds_axes()
#     pl.add_text("lsdf",position='lower_left',color='black')
    pl.update_bounds_axes()
#     pl.show_bounds(axes_ranges=[0, .1, 0, 1, 0, 2], show_zlabels=True)
    pl.save_graphic(m + '_' + models[i] + '_' + '.svg')
    
pl.show(interactive=False)


# pv.global_theme.transparent_background = True
# _ = pl.add_mesh(mesh,cmap=mpl.colormaps['inferno'])

# pl.set_viewup([0,1,0])
# pl.set_background('FFFFFFFF')
# pl.remove_scalar_bar()
# pl.remove_legend()
# pl.remove_floors()
# # pl.show_boundsounds()
# pl.view_xz()
# pl.show()
# pl.save_graphic(m + '_' + models[i] + '_' + '.svg')

In [ ]:
from svgutils.compose import *

svggg = 'gpuMemoryUsed.svg'
Figure("16cm", "6.5cm", 
        Panel(
              SVG(svggg),
              Text("A", 25, 20, size=12, weight='bold')
             ),
        Panel(
              SVG(svggg).scale(0.5),
              Text("B", 25, 20, size=12, weight='bold')
             ).move(280, 0)
        ).save("fig_final_compose.svg")

In [ ]:
import svgutils.compose as sc
from IPython.display import SVG # /!\ note the 'SVG' function also in svgutils.compose
import numpy as np
svg1= 'white_corticalflow_.svg'
svg2 = 'white_deepcsr_.svg'


# drawing a random figure on top of your SVG
# here starts the assembling using svgutils 


sc.Figure("16cm", "8cm", 
    sc.SVG(svg1) 
    )

## Testing Trimesh Save Mesh

In [1]:
import trimesh

In [2]:
 help(trimesh.exchange.export.export_mesh)

Help on function export_mesh in module trimesh.exchange.export:

export_mesh(mesh, file_obj, file_type=None, resolver=None, **kwargs)
    Export a Trimesh object to a file- like object, or to a filename
    
    Parameters
    -----------
    file_obj : str, file-like
      Where should mesh be exported to
    file_type : str or None
      Represents file type (eg: 'stl')
    resolver : None or trimesh.resolvers.Resolver
      Resolver to write referenced assets to
    
    Returns
    ----------
    exported : bytes or str
      Result of exporter



In [ ]:
pl.save_graphic('h.svg',raster=False,painter=True)

In [ ]:
pl._save_image(pl.image,)

In [ ]:
pl.image

In [ ]:
dir(pl)

In [ ]:
os.listdir('./showSVG')

In [ ]:
pl.save_graphic('pial' + '.svg')

In [ ]:
pl._image_scale = 2

In [ ]:
help(pl.camera_set)

In [ ]:
dir(pl)

In [ ]:
pl.add_text(0,10,'sdf')

In [ ]:
pl.add_text("lsdf",position='lower_left')

In [ ]:
help(pl.add_text)

In [ ]:
help(pl.update_bounds_axes)

## test svgutil

In [ ]:
pialfiles

In [ ]:
whitefiles

In [ ]:
models1

In [ ]:
models2

### Plot Pial

In [25]:
import svgutils.transform as sg
import sys 

#create new SVG figure
figure = sg.SVGFigure("20cm", "6.5cm")

# load matpotlib-generated figures
prefix = './pialpng/svg/1/1-'
# models = models1

start = 150
x0 = 100 + start
xend = 1000 + start
xpos = list(np.linspace(x0,xend,6))

plots = []
txts = []

modelnames = ['CortexODE','CorticalFlow','DeepCSR','PialNN','Vox2Cortex']
scale = .1

for i in range(5):
    fig = sg.fromfile(prefix + str(i+1) + '.svg')
    plot = fig.getroot()
    plot.rotate(90)
    plot.moveto(xpos[i], 0, scale, scale)
    plots.append(plot)
    
prefix = './whitepng/svg/2/2-'
for i in range(5):
    fig = sg.fromfile(prefix + str(i+1) + '.svg')
    plot = fig.getroot()
    plot.rotate(90)
    plot.moveto(xpos[i], 200, scale, scale)
    plots.append(plot)


sgh = 240
delta = -150

xpos = list(np.linspace(x0,xend+20,6))

txts = []
for i in range(5):
    if i!=3:
        txts.append(sg.TextElement(xpos[i] + delta,sgh, modelnames[i], size=12, weight="bold"))
        
txts.append(sg.TextElement(xpos[3] + delta,sgh-10, modelnames[3]+'*', size=12, weight="bold"))
txts.append(sg.TextElement(xpos[3] + delta,sgh+10, 'Topofit*', size=12, weight="bold"))

txts.append(sg.TextElement(10,120, 'Pial', size=12, weight="bold"))
txts.append(sg.TextElement(10,300, 'White', size=12, weight="bold"))

# legend

scale = .4
x = 560
y = 440
fig = sg.fromfile('legend.svg')
plot = fig.getroot()
plot.moveto(x, y,scale,scale)
plots.append(plot)
txts.append(sg.TextElement(x+140 , y - 10 , 'Vertex Distance / Mesh Diag (%)', size=12))

# append plots and labels to figure
figure.append(txts)
figure.append(plots)


# save generated SVG files
figure.save("fig_final02.svg")

In [23]:
import matplotlib.pyplot as plt
figure()

TypeError: 'SVGFigure' object is not callable

### Plot White

In [ ]:
#create new SVG figure
fig = sg.SVGFigure("20cm", "6.5cm")

prefix = 'white'
# load matpotlib-generated figures
fig1 = sg.fromfile(prefix + '_cortexode_.svg')
fig2 = sg.fromfile(prefix + '_corticalflow_.svg')
fig3 = sg.fromfile(prefix + '_deepcsr_.svg')
fig4 = sg.fromfile(prefix + '_topofit_.svg')
fig5 = sg.fromfile(prefix + '_vox2cortex_.svg')

# get the plot objects
plot1 = fig1.getroot()
plot2 = fig2.getroot()
plot3 = fig3.getroot()
plot4 = fig4.getroot()
plot5 = fig5.getroot()

x0 = -50
xend = 1500
xpos = list(np.linspace(x0,xend,6))

plot1.moveto(xpos[0], 0)
plot2.moveto(xpos[1], 0)
plot3.moveto(xpos[2], 0)
plot4.moveto(xpos[3], 0)
plot5.moveto(xpos[4], 0)



# add text labels
sgh = 120
delta = 30
txt0 = sg.TextElement(25 + delta,sgh, "a", size=12, weight="bold")
# txt2 = sg.TextElement(305 + delta,sgh, "b", size=12, weight="bold")
# txt3 = sg.TextElement(605 + delta,sgh, "c", size=12, weight="bold")
# txt4 = sg.TextElement(905 + delta,sgh, "d", size=12, weight="bold")
# txt5 = sg.TextElement(1205 + delta, sgh, "e", size=12, weight="bold")

sgh = 480
delta = 90
x0 = 25
xend = 1500
interval = 250
xpos = list(np.linspace(x0,xend,6))

models = ['CortexODE','CorticalFlow','DeepCSR','Topofit','Vox2Cortex']

txt1 = sg.TextElement(xpos[0] + delta,sgh, models[0], size=12, weight="bold")
txt2 = sg.TextElement(xpos[1] + delta,sgh, models[1], size=12, weight="bold")
txt3 = sg.TextElement(xpos[2] + delta,sgh, models[2], size=12, weight="bold")
txt4 = sg.TextElement(xpos[3] + delta,sgh, models[3], size=12, weight="bold")
txt5 = sg.TextElement(xpos[4] + delta,sgh, models[4], size=12, weight="bold")

# append plots and labels to figure
fig.append([plot1, plot2,plot3,plot4,plot5])
fig.append([txt0, txt1, txt2, txt3, txt4, txt5])

# save generated SVG files
fig.save("fig_final_white.svg")

## Convert PNG to SVG

In [ ]:
whitefiles

In [ ]:
!pip install potrace

In [ ]:
from potrace import Bitmap

from PIL import Image
import numpy as np

img = Image.open('whitevox2cortex.png')
data = np.array(img)
bitmap = Bitmap(data)
path = bitmap.trace()

## trim svg

In [ ]:
os.listdir('./showSVG')

In [ ]:
import vpype_cli as vp
vp.execute("read ./showSVG/pial_cortexode_.svg trim 30 20 write output.svg")

In [ ]:
dir(pl)

## save 1 image

In [ ]:

i = 0
mesh = mses1[0]
pl = pv.Plotter(window_size=(2000,2000),off_screen=False)

pv.global_theme.transparent_background = True
_ = pl.add_mesh(mesh,cmap=mpl.colormaps['inferno'])

pl.set_viewup([0,1,0])
pl.set_background('white')
pl.remove_scalar_bar()
pl.remove_legend()
pl.remove_floors()
if i == 3 and m=='pial':
    pl.view_xz()
# pl.show_boundsounds()
pl.show()
# filename = m + '_' + models[i] + '_' + '.png'

## Save SVG

In [ ]:
def save_svg(m = 'pial', models = models1, mses = mses1):
    datas = []
    for i in range(5):
        mesh = mses[i]
        pl = pv.Plotter(window_size=(2000,2000))
        
        pv.global_theme.transparent_background = True
        _ = pl.add_mesh(mesh,cmap=mpl.colormaps['inferno'])

        pl.set_viewup([0,1,0])
        pl.set_background('white')
        pl.remove_scalar_bar()
        pl.remove_legend()
        pl.remove_floors()
        if i == 3 and m=='pial':
            pl.view_xz()
        # pl.show_boundsounds()
        pl.show()
        filename = m + '_' + models[i] + '_' + '.png'
        data = pl.screenshot(filename, transparent_background=True, return_img=True,scale=4)
    return data
#         pl.save_graphic(m + '_' + models[i] + '_' + '.png')

In [ ]:
dir(pl)

In [ ]:
pl.save_graphic('test1.tex')

In [ ]:
help(pl.save_graphic)

In [ ]:
save_svg()

### check pl.screenshot for png

In [ ]:
data = pl.screenshot(filename, transparent_background=True, return_img=True, window_size=(600,1000),scale=4)

In [ ]:
save_svg('white',models2,mses2)

In [ ]:
dir(pl)

In [ ]:
import vpype_cli as vp
vp.execute("read white_deepcsr_.svg translate 300 400 trim 30 20 write output.svg")
# vp.execute("read  crop 0cm 1cm 1cm 2cm write output.svg")

In [ ]:
import svgwrite


In [ ]:
# Load the SVG file from disk
drawing = svgwrite.Drawing(filename=svg1)

# Do something with the drawing object, e.g. add shapes or text

# Save the modified SVG file to disk
drawing.save()

In [ ]:
dir(pl)


In [ ]:
svg1 = "pial_vox2cortex_.svg"
from svgpathtools import svg2paths, wsvg

# Load the SVG file and extract the paths
paths, attributes = svg2paths(svg1)

# Define the crop box (left, top, right, bottom)
crop_box = (100, 100, 400, 400)

# Crop each path using the crop box
cropped_paths = [path.crop(crop_box) for path in paths]

# Save the cropped paths to a new SVG file
wsvg(cropped_paths, filename='output.svg')


In [ ]:
svg1 = "pial_vox2cortex_.svg"
from svgpathtools import svg2paths, wsvg, path
paths, attributes, svg_attributes = svg2paths(svg1)
xmin, ymin, xmax, ymax = 100,100,200,200


In [ ]:
attributes

In [ ]:
svg_attributes

In [ ]:
paths

In [ ]:
new_paths = []
for p in paths:
    if p.bbox()[0] >= xmin and p.bbox()[1] >= ymin and p.bbox()[2] <= xmax and p.bbox()[3] <= ymax:
        new_paths.append(p)

wsvg(new_paths, filename='cropped_' + svg1)

In [ ]:
from svgutils.compose import *
from svgutils.transform import SVGFigure

# Define the size of the final SVG figure
width = 800
height = 600

# Create a new SVG figure with the specified size
fig = SVGFigure(width, height)

# Load the SVG files to be combined
svg1 = SVG(svg1)
svg2 = SVG(svg2)

# Create a new SVG object by stacking the individual SVGs vertically
combined_svg = vstack(svg1, svg2)

# Add the combined SVG object to the figure
fig.append(combined_svg)

# Save the figure to a new SVG file
fig.save('path/to/new/file.svg')



In [ ]:
import cv2

In [ ]:
from matplotlib import pyplot as plt
  
# create figure
fig = plt.figure(figsize=(10, 7))
  
# setting values to rows and column variables
rows = 2
columns = 2
  
# reading images
Image1 = cv2.imread('A.svg')
Image2 = cv2.imread('A1.svg')
Image3 = cv2.imread('A2.svg')
Image4 = cv2.imread('A3.svg')
  
# Adds a subplot at the 1st position
fig.add_subplot(rows, columns, 1)
  
# showing image
plt.imshow(Image1)
plt.axis('off')
plt.title("First")
  
# Adds a subplot at the 2nd position
fig.add_subplot(rows, columns, 2)
  
# showing image
plt.imshow(Image2)
plt.axis('off')
plt.title("Second")
  
# Adds a subplot at the 3rd position
fig.add_subplot(rows, columns, 3)
  
# showing image
plt.imshow(Image3)
plt.axis('off')
plt.title("Third")
  
# Adds a subplot at the 4th position
fig.add_subplot(rows, columns, 4)
  
# showing image
plt.imshow(Image4)
plt.axis('off')
plt.title("Fourth")

In [ ]:
mesh = mses1[0]
mapping = np.linspace(mesh['values'].min(), mesh['values'].max(), 256)
newcolors = np.empty((256, 4))
len(newcolors[mapping >= 1])

blue = np.array([12 / 256, 238 / 256, 246 / 256, 1.0])
black = np.array([11 / 256, 11 / 256, 11 / 256, 1.0])
grey = np.array([189 / 256, 189 / 256, 189 / 256, 1.0])
yellow = np.array([255 / 256, 247 / 256, 0 / 256, 1.0])

newcolors[mapping >= 1] = [blue] * len(newcolors[mapping >= 1])

In [ ]:
# Define the colors we want to use

mapping1 = np.linspace(0, 0.3, 100)
mapping2 = np.linspace(0.3, 1, 100)
mapping3 = np.linspace(1,2.4, 56)
mapping = list(mapping1) + list(mapping2) + list(mapping3)
colors = cc.bgy

mesh = mses1[0]
blue = np.array([12 / 256, 238 / 256, 246 / 256, 1.0])
black = np.array([11 / 256, 11 / 256, 11 / 256, 1.0])
grey = np.array([189 / 256, 189 / 256, 189 / 256, 1.0])
yellow = np.array([255 / 256, 247 / 256, 0 / 256, 1.0])


reds = [[244,143,177],[236,64,122],[216,27,96],[136,14,79]]
blues = [[144,202,249],[66, 165, 245 ],[25, 118, 210 ],[13, 71, 161]]
greens = [[200, 230, 201 ],[129, 199, 132],[76, 175, 80],[46, 125, 50]]

reds = [[i[0]/256,i[1]/256, i[2]/256, 1] for i in reds]
blues = [[i[0]/256,i[1]/256, i[2]/256, 1] for i in blues]
greens = [[i[0]/256,i[1]/256, i[2]/256, 1] for i in greens]

# red = np.array([1.0, 0.0, 0.0, 1.0])

mapping = np.linspace(mesh['values'].min(), mesh['values'].max(), 256)
newcolors = np.empty((256, 4))

# 199, 57, 17
# n = len(newcolors[.3<mapping < 1]) 
# print(n)
newcolors[mapping >= 1] = 50*[reds[0]] + 50* [reds[1]] + 50*[reds[2]] + 49*[reds[3]]
newcolors[mapping < 1] =  [reds[0]]*17 + 10*[blues[0]] + 10* [blues[1]] + 10*[blues[2]] + 10 *[blues[3]]
newcolors[mapping < .3] = 4*[greens[0]] + 4*[greens[1]] + 4*[greens[2]] + 5*[greens[3]] 

my_colormap = ListedColormap(newcolors)

# Make the colormap from the listed colors
import pyvista as pv
pl = pv.Plotter()
_ = pl.add_mesh(mesh,cmap=my_colormap)
# pl.camera_position([0,1,0])
pl.set_viewup([0,1,0])
dir(pl)
pl.show()

# my_colormap = ListedColormap(newcolors)

# mesh.plot(scalars='values', cmap=my_colormap)

In [ ]:
print('Results using get_hausdorff_distance')
d

In [ ]:
arr = ms[1].vertex_scalar_array()
print('results using compute_scalar_by_distance_from_another_mesh_per_vertex')
print('\nmin',min(abs(arr)))
print('max',max(abs(arr)))
print('mean',np.mean(abs(arr)))

In [ ]:
d = calc_hd()

In [ ]:
d.values()

In [ ]:
t = tmps[0]
import numpy as np
np.percentile(t, 2)

In [ ]:
np.percentile(t, 98)

In [ ]:
ms2.compute_scalar_by_distance_from_another_mesh_per_vertex()
ms2.compute_color_from_scalar_per_vertex()

In [ ]:
ms2[0].vertex_color_matrix()

In [ ]:
ms2[1].vertex_color_matrix()

In [ ]:
ms[2].plot(scalars='values')

In [ ]:
ms2[1]

In [ ]:
ms = ms2[1]

In [ ]:
ms

In [ ]:
dir(ms)

In [ ]:
ms2.show_polyscope()

In [ ]:
len(ms2[1].vertex_scalar_array())

In [ ]:
ms2[1].vertex_color_matrix()

In [ ]:
tmp = ms2[1].vertex_scalar_array()
max(tmp)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(tmp)

In [ ]:
res = []
for i in tmp:
    if i < 0:
        res.append([1,1,1,1])
    else:
        res.append([0,0,0,0])
        

In [ ]:
for mesh in ms2:
    mesh.set_color_map()

In [ ]:
dir(ml)

In [ ]:
print(ml.get_version())

In [ ]:
ml.apply_colormap

In [ ]:
dir(ms2[1].vertex_color_matrix())